In [1]:
!pip install pygaggle --quiet
!git clone --recursive https://github.com/castorini/pygaggle.git --quiet
!pip install -r pygaggle/requirements.txt --quiet
!pip install jsonlines --quiet
!pip install spacy==2.3.0 --quiet
!pip install torch==1.10.0 --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 2.0.0 requires transformers<5.0.0,>=4.6.0, but you have transformers 4.0.0 which is incompatible.
fatal: destination path 'pygaggle' already exists and is not an empty directory.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pygaggle 0.0.3.1 requires pydantic==1.5, but you have pydantic 1.7.4 which is incompatible.
pygaggle 0.0.3.1 requires pyserini==0.10.1.0, but you have pyserini 0.14.0 which is incompatible.
pygaggle 0.0.3.1 requires sentencepiece==0.1.94, but you have sentencepiece 0.1.95 which is incompatible.
pygaggle 0.0.3.1 requires spacy==2.2.4, but you have spacy 3.0.6 which is incompatible.
pygaggle 0.0.3.1 requires tokenizers==0.9.4, but you have tokeniz

In [2]:
# IMPORTS
#!pip install pygaggle
#!pip install -q torch==1.4.0 -f https://download.pytorch.org/whl/cu101/torch_stable.html
#!pip install transformers==4.11.3.

import torch

if torch.cuda.is_available(): 
    dev = "cuda:0"
    print(dev, torch.cuda.get_device_name(0))
    device = torch.device(dev)
else: 
    dev = "cpu"
    print(dev) 

from pyserini.search import SimpleSearcher
from pygaggle.rerank.base import hits_to_texts
from pygaggle.rerank.base import Query, Text
from pygaggle.rerank.transformer import MonoT5
from transformers import T5ForConditionalGeneration
import json
from pyserini.search import SimpleSearcher
import spacy
from sklearn.metrics import (accuracy_score, f1_score, classification_report)
import jsonlines
import os
from statistics import mean
import statistics
import math
from tqdm.notebook import tqdm
import numpy as np

import pandas as pd


#import pygaggle
#from pygaggle.rerank.base import Query, Text, hits_to_texts
#from pygaggle.rerank.transformer import MonoT5

#from pyserini.search import SimpleSearcher
#import pandas as pd

cpu


2021-12-22 08:50:23.593530: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-22 08:50:23.593570: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Load the data

In [3]:
def load_data(cleaned=True, accuracy=True):
    df = pd.read_json("./output/json_collection/complete_collection.json", orient='records', encoding='utf8')
        
    if cleaned:
        if accuracy:
            nlp = en_core_web_trf.load(disable=["parser", "ner"])
        else:
            nlp = en_core_web_sm.load(disable=["parser", "ner"])
        nlp.max_length = 2000000
        
        df['cleaned'] = df["contents"].apply(lambda x: prune_tokens(x, nlp))
        #df["contents"] = df["cleaned"].progress_apply(lambda x:re.split('\n',x))
    df["text_sent"] = df["contents"]
    return df

def load_queries():
    queries = dict()
    with open("./output/queries.txt", "r") as f:
        for line in f:
            cols = line.split("\t")
            queries[cols[0].strip()] = cols[1].strip()
    return queries

Define the reranker

In [ ]:
def reranker(queries, BM25=True, n_hits=10000, run='context_emb', it="Q0", index_path = './output/indexes/lucene_index_json'):
    # Define reranker model
    reranker =  MonoT5()
    
    # Define pyserini index
    #searcher = SimpleSearcher.from_prebuilt_index(index_path)
    searcher = SimpleSearcher(index_path)
    
    results = []
    for q_id in queries:
        print(q_id)
        query = Query(queries[q_id])
        hits = searcher.search(query.text, n_hits)
        
        texts = hits_to_texts(hits)
        #for i in range(0, 10):
        #    print(f'{i+1:2} {texts[i].metadata["docid"]:15} {texts[i].score:.5f} {texts[i].text}')
        #texts = [ Text(p[1], {'docid': p[0]}, 0) for p in hits]
        reranked = reranker.rerank(query, texts)
        
        for i in range(0, min(len(reranked), n_hits)):
            d = reranked[i].metadata["docid"]
            score = reranked[i].score
            results.append([str(q_id), str(it), str(d), str(i+1), str(score), str(run)])
            
            
        #for i in range(0, 10):
        #    print(f'{i+1:2} {reranked[i].metadata["docid"]:15} {reranked[i].score:.5f} {reranked[i].text}')
    check = set()
    for res in results:
        if ((res[0], res[2])) in check:
            raise Exception("Error: Duplicate query-doc is found", res[0], res[2])
        check.add((res[0], res[2]))

    output_str = "\n".join([' '.join(i) for i in results])
    open("./output/results/{}.run".format(run), "w").write(output_str)            
    


reranker(load_queries(), run="context_emb_content_index_10000")

301


/root/anaconda3/envs/IR4/lib/python3.7/site-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
